In [66]:
import pandas as pd
import numpy as np

def short_rate_tree(n,init_rate,u,d):
    tree = np.zeros((n+1,n+1))
    for i in range(n+1):
        for j in range(i+1):
            tree[i][j]=r00*u**j*d**(i-j)
    rate_df = pd.DataFrame(data=tree, columns=np.arange(tree.shape[0]))        
    return rate_df

def hazard_rate_tree(a,b,n):
    tree = np.zeros((n+1,n+1))
    for i in range(n):
        for j in range(i+1):
            tree[i,j]= a*b**(j-i/2)
    hazard_rate_df = pd.DataFrame(data=tree, columns=np.arange(tree.shape[0]))
    return hazard_rate_df

def defaultable_bond_price(short_rate, hazard_rate, face_value=100, R = 0.2):
    
    n = short_rate.shape[0]-1
    rate_tree = short_rate.to_numpy()
    hazard_tree = hazard_rate.to_numpy()
    payoff = np.ones((n+1,n+1))*face_value
    payoff[np.triu_indices(payoff.shape[0], 1)] = 0
#     print(payoff)
    
    for i in range(n-1,-1,-1):
        for j in range(i+1):
#             print(rate_tree[i,j], payoff[i,j], hazard_tree[i,j])
            payoff[i,j]=(q*(1-hazard_tree[i,j])*payoff[i+1,j+1]+(1-q)*(1-hazard_tree[i,j])*payoff[i+1,j])/(1+rate_tree[i,j])+ (q*hazard_tree[i,j]*R*face_value+(1-q)*hazard_tree[i,j]*R*face_value)/(1+rate_tree[i,j])
    
    df = pd.DataFrame(data=payoff, columns=np.arange(payoff.shape[0]))
    return df

def swap_price_lattice(short_rate,notional=1000000.0,fixed_rate = 0.045):
    n = short_rate.shape[0]-1
    rate_tree= short_rate.to_numpy()
    
    payoff = np.zeros((n+1,n+1))
    for i in range(n+1):
        payoff[n,i]=(rate_tree[n,i]-fixed_rate)/(1+rate_tree[n,i])
    for i in range(n-1,-1,-1):
        for j in range(i+1):
            payoff[i,j]= (rate_tree[i,j]-fixed_rate+q*payoff[i+1,j+1]+(1-q)*payoff[i+1,j])/(1+rate_tree[i,j])
    df = pd.DataFrame(data=payoff, columns=np.arange(payoff.shape[0]))
    return df

def swaption_price(swap_df,notional_principal = 1000000.0,fixed_rate = 0.045,t=5,K=0.0):
    lattice = swap_price_lattice(n,r00,u,d,q=.5,fixed_rate = 0.045)
    rate_tree= short_rate_tree(n,r00,u,d)
    f= np.vectorize(lambda x: max(x-K,0.0))
    payoff = f(lattice)[0:t+1,0:t+1]
    for i in range(t-1,-1,-1):
        for j in range(i+1):
            payoff[i,j]=(q*payoff[i+1,j+1]+(1-q)*payoff[i+1,j])/(1+rate_tree[i,j])
    return payoff[0,0]*notional_principal

### Question 1
| Period| Spot Rate |
| :- | :- |
| 1 | 3.0% |
| 2 | 3.1% |
| 3 | 3.2% |
| 4 | 3.3% |
| 5 | 3.4% |
| 6 | 3.5% |
| 7 | 3.55% |
| 8 | 3.6% |
| 9 | 3.65% |
| 10 | 3.7% |


As in the video modules, these interest rates assume per-period compounding. For example, the market-price of a zero-coupon bond that matures in period 6 is $Z_{0}^6 = \frac{100}{(1+0.035)^6} = 81.35$
assuming a face value of 100.
Assume $b=0.05$ is a constant for all i in the BDT model as we assumed in the video lectures. Calibrate the $a_i$  parameters so that the model term-structure matches the market term-structure. Be sure that the final error returned by Solver is at most $10^{-8}$. (This can be achieved by rerunning Solver multiple times if necessary, starting each time with the solution from the previous call to Solver.)

Once your model has been calibrated, compute the price of a payer swaption with notional $1M$ that expires at time $t=3$ with an option strike of 0. You may assume the underlying swap has a fixed rate of 3.9%% and that if the option is exercised then cash-flows take place at times $t = 4, ..., 10$. (The cash-flow at time t=i is based on the short-rate that prevailed in the previous period, i.e. the payments of the underlying swap are made in arrears.)

Submission Guideline: Give your answer rounded to the nearest integer. For example, if you compute the answer to be 10,456.67, submit 10457.



In [ ]:
# The answer for this questionc came from the "Term_Structure_Lattices_Assignment.xlsx" tab BDTQuestion1
b = 0.05
notional = 1000000
print(f"The price of a Payer Swaption with b={b} is 4101)

In [67]:
r00 = 0.05
u = 1.1
d = 0.9
q = 1/2
n = 10
short_df = short_rate_tree(n=n, init_rate=r00, u=u, d=d)
short_df

,0,1,2,3,4,5,6,7,8,9,10
0,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.045000,0.055000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.040500,0.049500,0.060500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.036450,0.044550,0.054450,0.066550,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.032805,0.040095,0.049005,0.059895,0.073205,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.029525,0.036086,0.044105,0.053906,0.065885,0.080526,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.026572,0.032477,0.039694,0.048515,0.059296,0.072473,0.088578,0.000000,0.000000,0.000000,0.000000
7,0.023915,0.029229,0.035725,0.043663,0.053366,0.065226,0.079720,0.097436,0.000000,0.000000,0.000000
8,0.021523,0.026306,0.032152,0.039297,0.048030,0.058703,0.071748,0.087692,0.107179,0.000000,0.000000
9,0.019371,0.023676,0.028937,0.035367,0.043227,0.052833,0.064573,0.078923,0.096461,0.117897,0.000000


### Question 3

Please refer to the material on defaultable bonds and credit-default swaps (CDS) to answer this question.

Construct a n=10-period binomial model for the short-rate, $r_{i,j} (i=0,1,2....9)$. The lattice parameters are: 
$r_{0,0} = 5\%, u = 1.1, d= 0.9$ and $q = 1 -q = 1/2$. This is the same lattice that you constructed in Assignment 5.

Assume that the 1-step hazard rate in node $(i, j)$ is given by $h_{i,j} = ab^{j - \frac{i}{2}}$ where $a = 0.01$ and $b = 1.01$. Compute the price of a zero coupon bond with FaceValue $F = 100$ and recovery $R = 20\%$

Submission Guideline: Give your answer rounded to two decimal places. For example, if you compute the answer to be 73.2367, submit 73.24.

In [68]:
a, b = 0.01, 1.01
hazard_rate_df = hazard_rate_tree(a=a, b=b, n=n)
hazard_rate_df

,0,1,2,3,4,5,6,7,8,9,10
0,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,0.009950,0.010050,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
2,0.009901,0.010000,0.010100,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,0.009852,0.009950,0.010050,0.010150,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
4,0.009803,0.009901,0.010000,0.010100,0.010201,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
5,0.009754,0.009852,0.009950,0.010050,0.010150,0.010252,0.000000,0.000000,0.000000,0.000000,0.0
6,0.009706,0.009803,0.009901,0.010000,0.010100,0.010201,0.010303,0.000000,0.000000,0.000000,0.0
7,0.009658,0.009754,0.009852,0.009950,0.010050,0.010150,0.010252,0.010354,0.000000,0.000000,0.0
8,0.009610,0.009706,0.009803,0.009901,0.010000,0.010100,0.010201,0.010303,0.010406,0.000000,0.0
9,0.009562,0.009658,0.009754,0.009852,0.009950,0.010050,0.010150,0.010252,0.010354,0.010458,0.0


In [78]:
face_value = 100
recovery_rate = 0.2
df = defaultable_bond_price(short_rate=short_df, hazard_rate=hazard_rate_df, face_value=face_value, R=recovery_rate)
print(f"The price of Zero Coupon Bond with Face Value {face_value} and Recovery Rate of {recovery_rate*100}% is {np.round(df.iloc[0,0], 2)}")
df

The price of Zero Coupon Bond with Face Value 100 and Recovery Rate of 20.0% is 57.22


,0,1,2,3,4,5,6,7,8,9,10
0,57.216858,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,63.033661,57.931391,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
2,68.594939,64.067438,59.002642,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,73.837924,69.935606,65.496265,60.517074,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
4,78.719960,75.463591,71.705254,67.415975,62.586554,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
5,83.216680,80.601845,77.546548,74.007330,69.950353,65.358296,0.000000,0.000000,0.000000,0.000000,0.0
6,87.319376,85.321669,82.962979,80.195897,76.975121,73.261819,69.029469,0.000000,0.000000,0.000000,0.0
7,91.032042,89.612299,87.921250,85.916105,83.551768,80.782895,77.567032,73.869024,0.000000,0.000000,0.0
8,94.368423,93.477553,92.408665,91.129866,89.605501,87.796631,85.662049,83.160009,80.250867,0.000000,0.0
9,97.349276,96.932438,96.429285,95.822847,95.093386,94.218200,93.171565,91.924860,90.446996,88.705248,0.0


In [71]:
df = swap_price_lattice(short_rate=short_df, notional=1000000,fixed_rate=0.045)
df

,0,1,2,3,4,5,6,7,8,9,10
0,0.038136,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-0.002269,0.072354,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,-0.034831,0.030090,0.102578,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,-0.059296,-0.004187,0.058345,0.128223,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,-0.075683,-0.030133,0.022286,0.081858,0.148555,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,-0.084232,-0.047710,-0.005162,0.043908,0.099823,0.162627,0.000000,0.000000,0.000000,0.000000,0.000000
6,-0.085355,-0.057132,-0.023902,0.014914,0.059826,0.111204,0.169191,0.000000,0.000000,0.000000,0.000000
7,-0.079583,-0.058807,-0.034121,-0.004968,0.029213,0.068941,0.114640,0.166559,0.000000,0.000000,0.000000
8,-0.067521,-0.053281,-0.036229,-0.015901,0.008204,0.036607,0.069818,0.108299,0.152405,0.000000,0.000000
9,-0.049810,-0.041186,-0.030791,-0.018301,-0.003345,0.014482,0.035623,0.060535,0.089672,0.123449,0.000000
